# Material de apoio {.unnumbered}

Até o momento não há material de apoio para esse tópico

# Scraping - Monitorando o cumprimento de uma decisão judicial

Algumas vezes, não temos a sorte de contarmos com uma base de dados estruturada que nos fornece as informações que desejamos ou precisamos de maneira fácil, como ocorreu nas duas aulas anteriores. Em certas ocasiões, somos obrigados a extrair os dados diretamente de um website e estrutura-los nós mesmos. Essa aula oferece uma introdução a essa técnica.

## O caso

Lembrando nossa motivação: um famoso político fluminense te procurou para ingressar com ação indenizatória combinada com pedido de obrigação de não fazer contra uma série de blogs por conta do conteúdo alegadamente difamatório que vem sendo postado contra ele. Esses dados não estão disponíveis em nenhuma API e em nenhum arquivo CSV a qual tenhamos acesso. Se queremos saber se nosso cliente está sendo citado, com que frequência, qual o conteúdo dos posts, e assim por diante, precisaremos acessar esses sites de alguma maneira. Poderíamos fazer isso manualmente, mas essa solução é cara, pouco prática e não pode ser aplicada para um conjunto grande de clientes e/ou blogs. Assim, a solução é usar **web scraping**, ou seja, *raspar* os sites que podemos acessar normalmente em busca de informações e armazenar essas informações em estruturas de dados que conhecemos.

Para entendermos o que é o **web scraping**, precisamos entender um pouco melhor como funcionam as páginas na internet e o que determina o seu conteúdo.

## O que é uma página na internet?

Uma página na internet é formada por uma série de arquivos que um determinado servidor fornece ao nosso navegador como resposta a uma requisição. A maneira como fazemos requisições e o formato das respostas são definidos por um protocolo: HTTP (ou HTTPS) - Hypertext Transfer Protocol (Secure). Saber, em linhas gerais, o funcionamento desse protocolo nos permite usa-lo para obter as informações que precisamos.

A primeira coisa que podemos fazer é enviar uma requisição `GET` para o servidor que queremos acessar. Geralmente fazemos isso digitando um endereço na barra de pesquisa do nosso navegador, mas hoje vamos acessar um site programaticamente via Python, com o pacote `requests`.

No caso, um dos blogs que nosso cliente hipotético deseja acionar é o [blog da Andréia Sadi](https://g1.globo.com/politica/blog/andreia-sadi/). Vamos ver como funcionam esses requests com base nesse blog.

In [1]:
import pandas as pd
import requests

sadi = requests.get("https://g1.globo.com/politica/blog/andreia-sadi/")

A linha acima simplesmente requisita aos servidores do G1 o blog da Andreia Sadi. Qual é a resposta dos servidores? Podemos verifica-la imprimindo `sadi`.

In [2]:
sadi

<Response [200]>

O texto acima não é muito informativo. Afinal, o que significa a resposta 200? Ela significa que deu tudo certo, mas isso não é suficiente para cumprirmos nenhum dos nossos objetivos. Precisamos conhecer o *conteúdo* do nosso request. Se acessarmos o blog a partir do nosso navegador, veremos algo com a seguinte cara:

<br>
<br>
    <img src=https://i.imgur.com/HpiDNo7.png style = "width:75%">
<br>
<br>

Em algum momento, precisamos entrar em cada um dos links que estão colocados na página acima e verificar o seu conteúdo. Para isso, precisamos ver algo vagamente parecido com isso no nosso código e trabalhar daí. Podemos fazer isso através do atributo `.content`:

In [4]:
sadi.content[:500]

b'<!DOCTYPE HTML><html lang="pt-br"> <head><meta charset="utf-8"><meta http-equiv="x-ua-compatible" content="ie=edge,chrome=1"><meta name="viewport" content="width=device-width, initial-scale=1"><script type="text/javascript" id="CDA_AAS">\n  window.cdaaas = window.cdaaas || {};\n  window.cdaaas.SETTINGS = window.cdaaas.SETTINGS || {};\n  window.cdaaas.PAGE_ANALYTICS_DATA = {"channel":"desktop","contentId":"9849cc53-01ed-4ea8-9ea6-7d605cbe2506","contentType":"home","serviceWorker":true};\n  window.cda'

O resultado não é o mesmo que veríamos em navegadores como o Google Chrome e o Firefox, mas possui todos os links e matérias que encontraríamos caso navegássemos ao endereço indicado através deles. Trata-se do código HTML utilizado para renderizar a página que vemos no navegador.

HTML é uma linguagem de programação muito mais simples e rudimentar do que Python. Não precisamos aprender HTML para fazer scraping, mas é importante sabermos como essa linguagem funciona. Todos os elementos de uma página escrita em HTML estão contidos em tags. A estrutura geral é: `<tag> seu código aqui </tag>`. Algumas vezes, para enriquecer a página com elementos estéticos ou outras informações, são passados *argumentos* para as tags. Assim, quando encontrarmos links veremos algo como `<a href='seu_link.com'> texto que fica azul </a>`. Podemos usar essas informações para transformar os dados da ilegível resposta dada pelo atributo `.content` em um conjunto estruturado de informações.

Um grande aliado na hora desse processo - chamado de *parsing* - é o pacote BeautifulSoup, que vamos importar abaixo.

In [7]:
from bs4 import BeautifulSoup

O BeautifulSoup tem como objetivo nos ajudar a parsear o conteúdo da nossa requisição. Como ele faz isso? Vamos criar uma variável chamada `sadi_soup` para ver como o pacote funciona.

In [13]:
sadi_soup = BeautifulSoup(sadi.content)


str(sadi_soup)[:3000]

'<!DOCTYPE HTML>\n<html lang="pt-br"> <head><meta charset="utf-8"/><meta content="ie=edge,chrome=1" http-equiv="x-ua-compatible"/><meta content="width=device-width, initial-scale=1" name="viewport"/><script id="CDA_AAS" type="text/javascript">\n  window.cdaaas = window.cdaaas || {};\n  window.cdaaas.SETTINGS = window.cdaaas.SETTINGS || {};\n  window.cdaaas.PAGE_ANALYTICS_DATA = {"channel":"desktop","contentId":"9849cc53-01ed-4ea8-9ea6-7d605cbe2506","contentType":"home","serviceWorker":true};\n  window.cdaaas.helpers = window.cdaaas.helpers || {};\n  window.cdaaas.internals = {};\n  window.HorizonClient = new Promise((resolve, reject) => { \n    window.cdaaas.internals.resolveHorizonPromise = resolve; \n    window.cdaaas.internals.rejectHorizonPromise = reject; \n  });\n\n  window.HorizonHelpers = {\n    unloadCallbacks: [],\n  };\n\n  window.cdaaas.featureFlags = [];\n  window.cdaaas.hasFF = (ff) => window.cdaaas.featureFlags.includes(ff);\n\n  window.glbDebug = window.cdaaas.debugger 

À primeira vista, o código ficou apenas marginalmente mais legível, mas outros métodos de objetos criados com o BeautifulSoup podem nos ajudar:

In [14]:
print(sadi_soup.prettify())

<!DOCTYPE HTML>
<html lang="pt-br">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge,chrome=1" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <script id="CDA_AAS" type="text/javascript">
   window.cdaaas = window.cdaaas || {};
  window.cdaaas.SETTINGS = window.cdaaas.SETTINGS || {};
  window.cdaaas.PAGE_ANALYTICS_DATA = {"channel":"desktop","contentId":"9849cc53-01ed-4ea8-9ea6-7d605cbe2506","contentType":"home","serviceWorker":true};
  window.cdaaas.helpers = window.cdaaas.helpers || {};
  window.cdaaas.internals = {};
  window.HorizonClient = new Promise((resolve, reject) => { 
    window.cdaaas.internals.resolveHorizonPromise = resolve; 
    window.cdaaas.internals.rejectHorizonPromise = reject; 
  });

  window.HorizonHelpers = {
    unloadCallbacks: [],
  };

  window.cdaaas.featureFlags = [];
  window.cdaaas.hasFF = (ff) => window.cdaaas.featureFlags.includes(ff);

  window.glbDebug = window.cdaaas.debugger = {
  

O método `.prettify()` nos ajuda a ver de maneira mais clara a estrutura do arquivo HTML que estamos usando. Ainda assim, parece que a esmagadora maioria do código não é útil para os nossos objetivos, controlando o comportamento dos menus da página, por exemplo.

Usar o inspetor de elementos do Google Chrome pode ser útil nessas horas. Se apertarmos com o botão direito do mouse em cima do primeiro link e pedirmos para o Chrome inspecionar o elemento, o próprio navegador nos indica onde está a categoria que nos interessa:

<br>
<br>
    <img src=https://i.imgur.com/K7q9XBp.png style = "width:75%">
<br>
<br>

Se observamos a imagem acima, veremos que o link para a matéria está dentro de uma tag do tipo `<div>` (que serve para separar diferentes elementos de uma página) de classe "feed-post-body". Podemos usar essas informações para fazer uma busca pelo primeiro post usando o BeautifulSoup:

In [15]:
primeiro_post = sadi_soup.find('div', class_ = 'feed-post-body')
print(primeiro_post.prettify())

<div class="feed-post-body">
 <div class="feed-post-header">
 </div>
 <div class="feed-post-body-title gui-color-primary gui-color-hover">
  <div class="_evt">
   <h2>
    <a class="feed-post-link gui-color-primary gui-color-hover" elementtiming="text-ssr" href="https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/10/leandro-grass-sera-presidente-do-iphan.ghtml">
     Leandro Grass será presidente do Iphan
    </a>
   </h2>
  </div>
 </div>
 <div class="feed-post-body-resumo" elementtiming="text-ssr">
  Sociólogo e ex-deputado distrital do DF vai chefiar órgão responsável por proteção do patrimônio histórico e artístico do país.
 </div>
 <div class="feed-media-wrapper">
  <a class="feed-post-figure-link gui-image-hover" href="https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/10/leandro-grass-sera-presidente-do-iphan.ghtml">
   <div class="bstn-fd-item-cover">
    <picture class="bstn-fd-cover-picture">
     <img class="bstn-fd-picture-image" elementtiming="image-ssr"

O código resultante ainda é longo e confuso, mas estamos nos aproximando do nosso objetivo: obter o link para o primeiro dos posts que aparecem no blog da Andréia Sadi. O método `.find()` toma como argumentos uma tag (no caso, `<div>`) e os argumentos opcionais que podem ser passados para essa tag. No caso, quisemos especificar que a tag estava definida como pertencendo à classe "feed-post-body".

    OBS: Tivemos que adicionar um sublinhado (_) depois de "class" na chamada, porque o Python reserva a palavra "class" internamente. Felizmente, os programadores por trás do BeautifulSoup já resolveram esse problema e podemos chamar "class_" para definir uma busca por uma classe específica em um código HTML.

Se olharmos o output das células anteriores, veremos que podemos separar o nosso código em alguns elementos. O BeautifulSoup sabe quebrar o código em diferentes elementos. Para fazermos isso, basta chamarmos o atributo `.children`, que retorna um gerador de uma lista. Para simplificar a explicação, basta usarmos ele em conjunto com a função `list()`:

In [16]:
list(primeiro_post.children)

[<div class="feed-post-header"></div>,
 <div class="feed-post-body-title gui-color-primary gui-color-hover"><div class="_evt"><h2><a class="feed-post-link gui-color-primary gui-color-hover" elementtiming="text-ssr" href="https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/10/leandro-grass-sera-presidente-do-iphan.ghtml">Leandro Grass será presidente do Iphan </a></h2></div></div>,
 <div class="feed-post-body-resumo" elementtiming="text-ssr">Sociólogo e ex-deputado distrital do DF vai chefiar órgão responsável por proteção do patrimônio histórico e artístico do país.</div>,
 <div class="feed-media-wrapper"><a class="feed-post-figure-link gui-image-hover" href="https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/10/leandro-grass-sera-presidente-do-iphan.ghtml"><div class="bstn-fd-item-cover"><picture class="bstn-fd-cover-picture"><img class="bstn-fd-picture-image" elementtiming="image-ssr" loading="eager" src="https://s2.glbimg.com/Xen9EIEiK-FoBAMW1Hen_hWRWBw=/540x304/t

O código acima nos permite ver 3 elementos diferentes. O link que estamos buscando está no segundo desses elementos:

In [17]:
list(primeiro_post.children)[1]

<div class="feed-post-body-title gui-color-primary gui-color-hover"><div class="_evt"><h2><a class="feed-post-link gui-color-primary gui-color-hover" elementtiming="text-ssr" href="https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/10/leandro-grass-sera-presidente-do-iphan.ghtml">Leandro Grass será presidente do Iphan </a></h2></div></div>

Se salvarmos esse elemento em uma variável própria, poderemos usa-lo para mostrar como acessar sintaticamente outras funcionalidades interessantes do BeautifulSoup.

In [18]:
primeiro_post_dados = list(primeiro_post.children)[1]

In [19]:
primeiro_post_dados.a

<a class="feed-post-link gui-color-primary gui-color-hover" elementtiming="text-ssr" href="https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/10/leandro-grass-sera-presidente-do-iphan.ghtml">Leandro Grass será presidente do Iphan </a>

Chamar o atributo `.a` é equivalente a fazer uma busca (`.find()`) pela tag `<a>`, usada para designar links:

In [20]:
primeiro_post_dados.find("a")

<a class="feed-post-link gui-color-primary gui-color-hover" elementtiming="text-ssr" href="https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/10/leandro-grass-sera-presidente-do-iphan.ghtml">Leandro Grass será presidente do Iphan </a>

Podemos conectar qualquer uma dessas sintaxes com a chamada ao atributo `.attrs`, que contém um dicionário com os elementos do link que identificamos:

In [21]:
primeiro_post_dados.a.attrs

{'href': 'https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/10/leandro-grass-sera-presidente-do-iphan.ghtml',
 'class': ['feed-post-link', 'gui-color-primary', 'gui-color-hover'],
 'elementtiming': 'text-ssr'}

Temos duas chaves nesse dicionário: 'href' e 'class'. Desde o começo, estamos buscando o link do primeiro post, portanto, basta chamarmos a chave 'href' como faríamos com qualquer outro dicionário:

In [22]:
primeiro_post_dados.a.attrs["href"]

'https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/10/leandro-grass-sera-presidente-do-iphan.ghtml'

Naturalmente, não queremos saber apenas do *primeiro* link: nosso cliente se importa com todas as menções ao seu nome. Para isso, ao invés de usar o método `.find()`, que retorna apenas o primeiro resultado, devemos usar o semelhante `.find_all()`:

In [23]:
todos_posts = sadi_soup.find_all('div', class_ = 'feed-post-body')

O resultado não é mais um único elemento, mas sim um conjunto iterável de elementos parecido com uma lista:

In [24]:
todos_posts

[<div class="feed-post-body"><div class="feed-post-header"></div><div class="feed-post-body-title gui-color-primary gui-color-hover"><div class="_evt"><h2><a class="feed-post-link gui-color-primary gui-color-hover" elementtiming="text-ssr" href="https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/10/leandro-grass-sera-presidente-do-iphan.ghtml">Leandro Grass será presidente do Iphan </a></h2></div></div><div class="feed-post-body-resumo" elementtiming="text-ssr">Sociólogo e ex-deputado distrital do DF vai chefiar órgão responsável por proteção do patrimônio histórico e artístico do país.</div><div class="feed-media-wrapper"><a class="feed-post-figure-link gui-image-hover" href="https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/10/leandro-grass-sera-presidente-do-iphan.ghtml"><div class="bstn-fd-item-cover"><picture class="bstn-fd-cover-picture"><img class="bstn-fd-picture-image" elementtiming="image-ssr" loading="eager" src="https://s2.glbimg.com/Xen9EIEiK-FoBAMW1He

Podemos gerar uma lista de links para os posts do blog da Andréia Sadi usando `todos_posts` em um for loop com a lógica que vimos acima. Afinal, podemos assumir sem muitos problemas que a estrutura usada para o primeiro post se replicará para todos os demais.

    OBS: Esse não é necessariamente o caso! Alguns sites colocam o primeiro post em destaque, por exemplo. Quando isso acontece, temos que lidar de forma diferente com o primeiro post e com os demais.

In [25]:
todos_links = []

for post in todos_posts:
    post_root = list(post.children)[1]
    todos_links.append(post_root.a.attrs["href"])
    
todos_links

['https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/10/leandro-grass-sera-presidente-do-iphan.ghtml',
 'https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/09/tcu-avalia-obrigar-golpistas-ibaneis-e-anderson-torres-a-pagar-por-prejuizos-de-depredacao-bolsonarista.ghtml',
 'https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/09/abin-alertou-autoridades-de-seguranca-publica-do-df-pela-manha-sobre-ameaca-de-invasao-bolsonarista.ghtml',
 'https://g1.globo.com/politica/blog/andreia-sadi/noticia/2023/01/09/pf-ve-falha-e-reforca-desconfianca-de-ala-do-gsi-alinhada-a-golpistas.ghtml',
 'https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/09/para-ministros-do-stf-afastamento-de-ibaneis-e-recado-para-politicos-omissos-com-golpistas.ghtml',
 'https://g1.globo.com/politica/blog/andreia-sadi/post/2023/01/08/ministro-da-defesa-diz-que-terrorismo-de-radicais-e-tentativa-de-golpe-nao-vamos-mais-aturar-isso.ghtml',
 'https://g1.globo.com/politica/blog/andreia-sadi/p

Bom! Agora temos todos os links da primeira página do blog da Andreia Sadi. Mas isso não nos diz muita coisa... O que precisamos fazer?

Repetir a exploração que usamos para descobrir os links para inspecionar o código HTML do primeiro post e, a partir disso, construir um for loop capaz de extrair o texto de cada post.

In [26]:
#faz a requisição do primeiro link
primeiro_post = requests.get(todos_links[0])

#transforma o conteúdo da resposta em um objeto BeautifulSoup
primeiro_post_soup = BeautifulSoup(primeiro_post.content)

#imprime a resposta de maneira organizada
print(primeiro_post_soup.prettify())

<!DOCTYPE HTML>
<html lang="pt-br">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge,chrome=1" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <script id="CDA_AAS" type="text/javascript">
   window.cdaaas = window.cdaaas || {};
  window.cdaaas.SETTINGS = window.cdaaas.SETTINGS || {};
  window.cdaaas.PAGE_ANALYTICS_DATA = {"adsCount":2,"channel":"desktop","contentId":"bf6759da-0a12-413b-ad3b-2444dd26a91b","contentType":"post","elementCount":9,"featuredElement":"","imageCount":1,"metadata":"wordCount=214\u0026wordCountRange=0 a 265\u0026comments=true\u0026summary=false","pageType":"multi-content","paragraphCount":8,"serviceWorker":true,"videoCount":0};
  window.cdaaas.helpers = window.cdaaas.helpers || {};
  window.cdaaas.internals = {};
  window.HorizonClient = new Promise((resolve, reject) => { 
    window.cdaaas.internals.resolveHorizonPromise = resolve; 
    window.cdaaas.internals.rejectHorizonPromise = reject; 
  })

De novo, é difícil encontrar o que queremos no código HTML da página. O ideal é repetirmos a mesma estratégia que usamos na homepage do blog:

<br>
<br>
    <img src=https://i.imgur.com/BiY0Lie.png style = "width:95%">
<br>
<br>

Dessa vez, podemos notar que o elemento que queremos - o texto do artigo - está dentro de uma tag específica: `article`. Podemos extrai-la usando o método `.find()`:

In [27]:
primeiro_texto = primeiro_post_soup.find("article")

print(primeiro_texto.prettify())

<article itemprop="articleBody">
 <meta content="true" itemprop="isAccessibleForFree"/>
 <div id="chunk-9i594">
  <div class="row medium-uncollapsed content-media content-photo" data-block-type="backstage-photo">
   <div class="mc-column content-media__container">
    <!-- Line Break for lightbox caption due to styling limitation -->
    <div class="content-media-container glb-skeleton-box" style="--skeleton-width: 100%; --skeleton-height: auto; padding-top: 56.25%">
     <figure class="content-media-figure">
      <amp-img alt="Leandro Grass, segundo colocado na eleição ao governo do DF em 2022. — Foto: TV Globo/Reprodução" class="content-media-image" height="1080" layout="responsive" lightbox="lightbox-amp-carousel" noloading="" src="https://s2.glbimg.com/bWTpk3fAMAFdB60d0-tOpAW4yN0=/0x0:1920x1080/984x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2022/O/r/4sUV8IQFixoZP0ljOhHA/leandro-grass-bddf.transfer-frame-245.jpeg" srcset=

Apesar de termos chegado muito mais perto, ainda há muito trabalho a ser feito: é difícil ler o texto do artigo da maneira como ele está colocado. Felizmente, o BeautifulSoup possui um método que nos permite contornar esse problema sem mais delongas: `.get_text()`:

In [28]:
primeiro_texto.get_text()

'           1 de 1\rLeandro Grass, segundo colocado na eleição ao governo do DF em 2022. — Foto: TV Globo/Reprodução     Leandro Grass, segundo colocado na eleição ao governo do DF em 2022. — Foto: TV Globo/Reprodução                O sociólogo Leandro Grass será o presidente do Iphan (Instituto do Patrimônio Histórico e Artístico Nacional), órgão responsável pela preservação do patrimônio material e imaterial brasileiro.\xa0        Grass é brasiliense e tem 37 anos. É mestre em Desenvolvimento Sustentável pela Universidade de Brasília (UnB), gestor cultural formado na Organização dos Estados Ibero-Americanos, da Espanha, e ex-pesquisador do Observatório de Políticas Públicas Culturais da UnB.       Em 2018, Grass foi eleito deputado distrital do DF, cargo que ocupou até 2022. Durante o mandato presidiu a Frente Parlamentar pela Promoção dos Direitos Culturais e foi vice-presidente da Comissão de Educação, Saúde e Cultura da Câmara Legislativa do Distrito Federal.             Grass ten

Missão cumprida! Com isso, poderíamos simplesmente criar uma lista com os textos de cada post e verificar iterativamente se o nome do nosso cliente consta em cada um deles e quantas vezes isso ocorre, mas essa é uma maneira pobre de resolver o nosso problema. Afinal, queremos mostrar para o juiz que as menções difamatórias são constantes, se repetem múltiplas vezes no tempo e queremos poder indicar de maneira precisa quais foram os posts que consideramos ofensivos. Portanto, faz sentido buscarmos por mais duas informações na página: o título de cada post e a data de sua publicação. Assim, podemos guardar as informações em um dicionário.

## Exercício

Para encontrar o título e a data do post, podemos seguir a mesma estratégia: inspecionar os elementos relevantes no nosso navegador e encontrar a tag apropriada usando o método `.find()`. Façam isso com relação ao primeiro post do blog da Andreia Sadi (naturalmente, o post vai ser diferente do que usamos como exemplo até agora, mas o código de correção leva isso em consideração). Guardem **o texto dos resultados** em duas variáveis chamadas `primeiro_titulo` e `primeira_data`, respectivamente.

In [29]:
#seu código aqui


In [ ]:
from correcoes import aula12_ex1
aula12_ex1(primeiro_titulo, primeira_data)

Agora que temos todas as informações necessárias, podemos criar um loop que expanda o nosso approach para todos os links da primeira página e guardar as informações em um dicionário.

In [31]:
sadi_posts = {}

for link in todos_links:
    #faz a requisição e passa o resultado para o Beautiful Soup
    post = requests.get(link)
    post_content = BeautifulSoup(post.content)
    
    #extrai os atributos que queremos guardar
    post_title = post_content.find('h1', class_ = 'content-head__title').get_text()
    post_date = post_content.find('time').get_text()
    post_text = post_content.find("article").get_text()
    
    #adiciona os atributos ao dicionário, usando o título como chave
    sadi_posts[post_title] = {'data' : post_date,
                             'texto' : post_text}
    
sadi_posts

{'Leandro Grass será presidente do Iphan ': {'data': ' 10/01/2023 13h15 ',
  'texto': '           1 de 1\rLeandro Grass, segundo colocado na eleição ao governo do DF em 2022. — Foto: TV Globo/Reprodução     Leandro Grass, segundo colocado na eleição ao governo do DF em 2022. — Foto: TV Globo/Reprodução                O sociólogo Leandro Grass será o presidente do Iphan (Instituto do Patrimônio Histórico e Artístico Nacional), órgão responsável pela preservação do patrimônio material e imaterial brasileiro.\xa0        Grass é brasiliense e tem 37 anos. É mestre em Desenvolvimento Sustentável pela Universidade de Brasília (UnB), gestor cultural formado na Organização dos Estados Ibero-Americanos, da Espanha, e ex-pesquisador do Observatório de Políticas Públicas Culturais da UnB.       Em 2018, Grass foi eleito deputado distrital do DF, cargo que ocupou até 2022. Durante o mandato presidiu a Frente Parlamentar pela Promoção dos Direitos Culturais e foi vice-presidente da Comissão de Educ

Pronto! Possuímos um dicionário com os últimos posts do blog da Andreia Sadi. Podemos procurar o nome do nosso cliente em cada um dos resultados usando a sintaxe `in` em um for loop. Vamos supor (pelo tema tratado no blog no exemplo de teste) que nosso cliente é Rodrigo Maia:

In [32]:
for key, value in sadi_posts.items():
    if 'Rodrigo Maia' in value['texto']:
        print(f"O post de título {key}, publicado em {value['data']} citou o cliente Rodrigo Maia")

Missão cumprida! Para finalizar essa aula, podemos guardar tudo em uma função `get_sadi_posts(nome_do_cliente)`, para verificarmos a evolução dos posts enquanto aguardamos o desfecho da ação judicial e para monitorar o cumprimento de eventual sentença.

In [33]:
def get_sadi_posts(cliente):
    sadi_response = requests.get("https://g1.globo.com/politica/blog/andreia-sadi/")
    sadi_soup = BeautifulSoup(sadi_response.content)
    todos_posts = sadi_soup.find_all('div', class_ = 'feed-post-body')

    todos_links = []

    for post in todos_posts:
        post_root = list(post.children)[1]
        todos_links.append(post_root.a.attrs["href"])
        
    sadi_posts = {}

    for link in todos_links:
        #faz a requisição e passa o resultado para o Beautiful Soup
        post = requests.get(link)
        post_content = BeautifulSoup(post.content)

        #extrai os atributos que queremos guardar
        post_title = post_content.find('h1', class_ = 'content-head__title').get_text()
        post_date = post_content.find('time').get_text()
        post_text = post_content.find("article").get_text()

        #adiciona os atributos ao dicionário, usando o título como chave
        sadi_posts[post_title] = {'data' : post_date,
                                 'texto' : post_text}
        
    for key, value in sadi_posts.items():
        if cliente in value['texto']:
            print(f"O post de título {key}, publicado em {value['data']} citou o cliente {cliente}")

## Desafio

Agora que aprendemos a fazer webscraping, podemos aplicar a mesma estratégia para fazer scraping do [blog do Paulo Henrique Amorim](https://www.conversaafiada.com.br/). Faça isso para, pelo menos, os **3 posts mais recentes** e guarde os resultados em um dicionário chamado `paulo_henrique_dict`. O dicionário deve ter a mesma estrutura do que criamos acima para o blog da Andreia Sadi:

`paulo_henrique_dict[post_title] = {'data' : post_date, 'texto' : post_text}`

DICA: Para acessar o atributo `class` de uma tag HTML, precisamos especificar, no método `.find()` um argumento chamado `class_`. Quando queremos acessar o atributo `property`, basta adicionarmos o argumento `property` à nossa chamada de `.find()`.

In [6]:
#seu código aqui


In [ ]:
from correcoes import aula12_desafio
aula12_desafio(paulo_henrique_dict)

## Resumo

Nessa aula, aprendemos sobre web scraping e sobre como podemos usar o Python e os pacotes requests e BeautifulSoup para extrair dados estruturados a partir de fontes de informação não estruturadas.

* Quando um servidor recebe uma requisição HTTP do tipo GET, ele envia, dentre outros arquivos, uma página da web escrita em HTML.

* HTML é uma linguagem cujo conteúdo vem demarcado por tags que se abrem e se fecham e que podem nos ajudar a identificar onde estão expressados os diferentes elementos de um site.

* Usando a função de inspecionar elementos, podemos, com relativa facilidade, encontrar as tags únicas que identificam os diferentes campos de um blog.

* O pacote BeautifulSoup nos permite procurar e extrair informações das tags HTML de maneira fácil.

# Bonus: modo easy

Algumas vezes, queremos simplesmente extrair os dados de uma tabela que se encontra, por exemplo, na Wikipedia. Por exemplo, [uma página da wikipedia oferece uma lista de constituições nacionais e suas datas de promulgação](https://en.wikipedia.org/wiki/List_of_national_constitutions). E se quisermos usar essa lista dentro do Python? Felizmente, podemos usar o Pandas para fazer isso usando a função `read_html`, que retorna uma *lista de dataframes* com todas as tabelas que estiverem contidas na página:

In [2]:
wiki_constitutions = pd.read_html("https://en.wikipedia.org/wiki/List_of_national_constitutions")
len(wiki_constitutions)

13

A lista contém 4 tabelas. Uma análise da página mostra apenas 3 tabelas contendo dados interessantes. Isso acontece porque parte do rodapé é identificado como uma tabela (o modo easy não é perfeito). A tabela que queremos pode ser acessada através do índice 0:

In [3]:
wiki_constitutions[0].head()

,State,Date ratified,Word count[1]
0,Constitution of Afghanistan,"October 1, 1964",10227
1,Constitution of Albania,"November 28, 1998",13826
2,Constitution of Algeria,"December 8, 1996",10038
3,Constitution of Andorra,"February 2, 1993",8740
4,Constitution of Angola,"January 21, 2010",27181
